In [ ]:
!echo -e "smartapi-python\npyotp\nlogzero\nFlask\ndatetime\npandas\nnumpy" > requirements.txt
# -e option enables interpretation of backslash escapes, allowing the newline character \n to be interpreted correctly.

In [ ]:
!pip3 install -r requirements.txt

In [4]:
# Define time range for candle data
from datetime import date, datetime 

today_date = date.today()
from_time = datetime(today_date.year, today_date.month, today_date.day, 9, 15).strftime('%Y-%m-%d %H:%M')
to_time = datetime(today_date.year, today_date.month, today_date.day, 15, 30).strftime('%Y-%m-%d %H:%M')
print(f"{from_time}--{to_time}")

2024-12-20 09:15--2024-12-20 15:30


In [1]:
'''flow 
# FLow based on data  functional flow   based on no of funciton based on abscvtraction of function 


symDict --> symDf --> processed+filteredNseDf --> obj.getCandleData(NseDf) --> 

jsonRes | apijsontodf(jsonRes) --> candleDf | singalGen(candleDf)


symUrl | jsonRes --> jsonRes | symDf --> 
symDf | candleDf --> canleDf | singal -->

# Flow based on script implentatino 
--> 

'''
# using 2space function name not 3 

# Env if any 
from envji import totpkey, apikey, usrn, pin

# Global Constants
TOTP_KEY = totpkey
API_KEY = apikey
USERNAME = usrn
PIN =  pin

# Flow Constants
CANDLE_INTERVAL = "ONE_MINUTE"
SYMBOL_LIMIT = 10
FROM_TIME = "2024-12-20 09:15"
TO_TIME = "2024-12-20 15:30"



# Gi
from logzero import logger
import time 


def auth():
# Fi
    from SmartApi import SmartConnect
    from pyotp import TOTP
    
    try: 
        obj = SmartConnect(API_KEY)
        obj.generateSession(USERNAME, PIN, TOTP(TOTP_KEY).now())
        logger.info("AUTH success")
        return obj
    except Exception as e:
        logger.error(f"AUTH failure: {e}")
        raise

def fetch_sym():
    import requests
    import pandas as pd 

    url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
    try:
        data = requests.get(url).json()
        df = pd.DataFrame.from_dict(data)
        df['expiry'] = pd.to_datetime(df['expiry'], errors='coerce')
        df = df.astype({'strike': float}, errors='ignore')
        filtered_df = df[
            (df['exch_seg'] == 'NSE') &
            df['symbol'].str.endswith('-EQ')
        ]
        return filtered_df
    except Exception as e:
        logger.error(f"Failed to fetch trading symbols: {e}")
        raise


def get_candles(obj, symbol_info, from_time, to_time):
    """
    Fetches candle data for a given symbol within the specified time range.
    """
    # Import used in this function
    import pandas as pd

    try:
        historic_param = {
            "exchange": symbol_info.exch_seg,
            "symboltoken": symbol_info.token,
            "interval": CANDLE_INTERVAL,
            "fromdate": from_time,
            "todate": to_time,
        }
        response = obj.getCandleData(historic_param)
        columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        df = pd.DataFrame(response['data'], columns=columns)
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S%z')
        df['symbol'] = symbol_info.symbol
        return df
    except Exception as e:
        logger.error(f"Failed to fetch candle data for {symbol_info.symbol}: {e}")
        return pd.DataFrame()

def process_df(dataframes):
    """
    Processes candle data to generate buy/sell signals based on the last row.
    """
    signals = []
    for df in dataframes:
        if not df.empty:
            last_row = df.iloc[-1]
            if last_row['close'] > last_row['open']:
                df['Signal'] = 2  # Buy signal
                signals.append(last_row['symbol'])
            elif last_row['close'] < last_row['open']:
                df['Signal'] = 1  # Sell signal
            else:
                df['Signal'] = None
        else:
            logger.warning("Empty DataFrame encountered.")
    return signals

def main():
    """
    Main function to orchestrate the fetching and processing of candle data.
    """
    # Authentication
    obj = auth()

    # Fetch trading symbols
    trading_symbols = fetch_sym()

    # Define time range for candle data
    from_time = FROM_TIME
    to_time = TO_TIME
    # Fetch and process candle data
    dataframes = []
    for i, row in trading_symbols.iterrows():
        if i >= SYMBOL_LIMIT:  # Limit to SYMBOL_LIMIT symbols
            break
        df = get_candles(obj, row, from_time, to_time)
        dataframes.append(df)
        time.sleep(0.4)  # Throttle API requests

    # Generate signals
    signals = process_df(dataframes)

    # Output results
    logger.info(f"Signals generated: {signals}")

if __name__ == "__main__":
    main()


ImportError: cannot import name 'usrn' from 'envji' (/home/acer/Desktop/Dec12/tvcharts/fin/service3/envji.py)